In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
lookback = 48

# 加载info_scaler
with open('info_scaler.pkl', 'rb') as f:
    info_scaler = pickle.load(f)

with open('price_scaler.pkl', 'rb') as f:
    price_scaler = pickle.load(f)

# 加载所支持的公司代码
with open('data/code_set.pkl', 'rb') as f:
    code_set = pickle.load(f)

# 加载股价字典
price_dict = dict()
for code in code_set:
    price_df = pd.read_csv('data/price/{}.csv'.format(code))
    price_df = price_df.drop(['trade_date', 'pre_close'], axis=1)
    price_dict[code] = price_df

In [3]:
# 加载公司基本面字典
company_dict = dict()
for code in code_set:
    company_df = pd.read_csv('data/fund/{}.csv'.format(code))
    company_df = company_df.drop(['ts_code', 'ann_date', 'end_date'], axis=1)
    company_dict[code] = company_df

# 加载家族企业
use = ['Symbol', 'ContrshrProportion', 'IsRelatedTrading', 'ShareholderFirstProp',
       'ControlProportion', 'FamEntyp_1.0', 'FamEntyp_2.0', 'FamEntyp_3.0',
       'BoardCode_P3401', 'BoardCode_P3402', 'BoardCode_P3403', 'FamStyle_1',
       'FamStyle_2', 'ManGeneration_1.0', 'ManGeneration_2.0',
       'FamNameStatus_1', 'FamNameStatus_2', 'FamNameStatus_3',
       'FamNameStatus_4', 'FamNameStatus_5']
fam = pd.read_csv('data/family_firm_clean.csv', index_col=0, usecols=use)

In [4]:
def transform_price(price_df, forecast_close_line):
    price_df['delta'] = price_df.apply(lambda x: x['close'] - forecast_close_line, axis=1)
    price_values = price_scaler.transform(price_df)
    return price_values[:lookback]

In [5]:
code = '600998.SH'
forecast_close_line = 12.1

In [6]:
price_df = price_dict[code]
price_values = transform_price(price_df, forecast_close_line)

In [7]:
company_df = company_dict[code].loc[0]

In [8]:
company_df

current_ratio         1.271000e+00
quick_ratio           9.808000e-01
cash_ratio            9.006000e-01
inv_turn              3.316400e+00
ar_turn               2.082100e+00
ca_turn               8.540000e-01
netprofit_margin      1.573500e+00
grossprofit_margin    8.379500e+00
roe                   4.055000e+00
debt_to_assets        6.928800e+01
op_yoy                3.562380e+01
ebt_yoy               3.522160e+01
assets_yoy            5.154000e-01
tr_yoy                1.408760e+01
or_yoy                1.408760e+01
q_gr_yoy              1.742770e+01
q_gr_qoq             -3.637100e+00
q_sales_yoy           1.742770e+01
q_sales_qoq          -3.637100e+00
q_op_yoy              3.975540e+01
q_op_qoq              5.208080e+01
q_profit_yoy          3.515150e+01
q_profit_qoq          4.458190e+01
revenue               4.842905e+10
Name: 0, dtype: float64

In [22]:
pledge_price = price_df.loc[0]['close']
data = pd.Series(np.array([code, pledge_price, forecast_close_line]),
                    index=['ts_code', 'pledge_price', 'forecast_close_line'])

In [23]:
data

ts_code                600998.SH
pledge_price             17.5891
forecast_close_line         12.1
dtype: object

In [24]:
data = pd.concat([data, company_df])

In [25]:
len(data.index)

27

In [30]:
df = data.to_frame().T

In [31]:
df

,ts_code,pledge_price,forecast_close_line,current_ratio,quick_ratio,cash_ratio,inv_turn,ar_turn,ca_turn,netprofit_margin,...,or_yoy,q_gr_yoy,q_gr_qoq,q_sales_yoy,q_sales_qoq,q_op_yoy,q_op_qoq,q_profit_yoy,q_profit_qoq,revenue
0,600998.SH,17.5891,12.1,1.271,0.9808,0.9006,3.3164,2.0821,0.854,1.5735,...,14.0876,17.4277,-3.6371,17.4277,-3.6371,39.7554,52.0808,35.1515,44.5819,4.8429e+10


In [32]:
df.merge(fam, how='left', left_on='ts_code', right_on='Symbol')

,ts_code,pledge_price,forecast_close_line,current_ratio,quick_ratio,cash_ratio,inv_turn,ar_turn,ca_turn,netprofit_margin,...,BoardCode_P3403,FamStyle_1,FamStyle_2,ManGeneration_1.0,ManGeneration_2.0,FamNameStatus_1,FamNameStatus_2,FamNameStatus_3,FamNameStatus_4,FamNameStatus_5
0,600998.SH,17.5891,12.1,1.271,0.9808,0.9006,3.3164,2.0821,0.854,1.5735,...,0,1,0,1,0,1,1,1,1,1
